Import libraries

In [1]:
import os 
from tqdm import tqdm
from pathlib import Path
from utils import load_pickle, pickle_data, load_json

Helper Functions

In [2]:
def get_CF(data_folder, data_ids):
    
    all_CFs = set()

    for id in tqdm(data_ids):

        data = load_pickle(data_folder / id)
        peaks = data["peaks"]
        f_pred = [f["comment"]["f_pred"] for f in peaks]
        f_pred = [f for f in f_pred if f != ""]
        all_CFs.update(f_pred)
    
    return all_CFs

Settings

In [32]:
data_folder = Path("/data/rbg/users/klingmin/projects/MS_processing/data")
splits_folder = Path("/data/rbg/users/klingmin/projects/MS_processing/data_splits")
CFs_folder = Path("/data/rbg/users/klingmin/projects/MS_processing/CFs")
if not os.path.exists(CFs_folder): os.makedirs(CFs_folder)

datasets = ["massspecgym", "nist2023"]
considered_splits = ["random", "scaffold_vanilla", "inchikey_vanilla"]

Question 1: is there a correlation between train-test CF overlap and jaccard score

Step 1: Consolidate the set of chemical formula

In [ ]:
for dataset in tqdm(datasets): 

    folder = data_folder / dataset
    frags_folder = folder / "frags_preds"
    splits = splits_folder / dataset / "splits"

    for s in tqdm(considered_splits):

        current_split = load_json(splits / f"{s}.json")
        CFs = CFs_folder / dataset/ f"{s}_split"

        if not os.path.exists(CFs):

            os.makedirs(CFs)
            print(f"Proceeding to get the set of CFs for dataset: {dataset}, split: {s}")
            
            train_ids, val_ids, test_ids = current_split["train"], current_split["val"], current_split["test"]
            train_CFs = get_CF(frags_folder, train_ids)
            val_CFs = get_CF(frags_folder, val_ids)
            test_CFs = get_CF(frags_folder, test_ids)

            pickle_data(train_CFs, CFs / "train_CFs.pkl")
            pickle_data(val_CFs, CFs / "val_CFs.pkl")
            pickle_data(test_CFs, CFs / "test_CFs.pkl")

  0%|          | 0/2 [00:00<?, ?it/s]

Proceeding to get the set of CFs for dataset: massspecgym, split: random


Step 2: Look at the overlap

In [31]:
for dataset in tqdm(datasets): 

    for s in tqdm(considered_splits):

        CFs = CFs_folder / dataset/ f"{s}_split"
        train_CFs = load_pickle(CFs / "train_CFs.pkl")
        test_CFs = load_pickle(CFs / "test_CFs.pkl")

        oov_test_CFs = test_CFs - train_CFs
        oov_test_CFs_rate = round(len(oov_test_CFs) / len(test_CFs) * 100, 3)

        print(s)
        print(len(train_CFs))
        print(len(test_CFs))
        print(len(oov_test_CFs), oov_test_CFs_rate)
        print() 


100%|██████████| 1/1 [00:00<00:00,  4.90it/s]

random
168863
79837
20871 26.142

scaffold_vanilla
168860
70059
16792 23.968

inchikey_vanilla
166293
85696
24985 29.155

